# Data Exploration

In [1]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('baseline')

In [2]:
def json_to_df(json_file=None):
    """ Reads a single JSON file containing the
    shipment information and converts it to a
    structured dataframe
    """
    import pandas as pd
    from pandas.io.json import json_normalize
    import numpy as np
    logger.debug("Converting to dataframe: {}".format(json_file))
    df = pd.read_json(json_file, dtype={"shipment_number": str})
    normalized_df = json_normalize(df['events'])
    df.drop(["events"], axis=1, inplace=True)
    df = pd.concat([df, normalized_df], axis=1)
    if "Die Sendung wurde elektronisch angeku00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen." == df.tail(1)["status"].item():
        logger.warning("Final delivery state is strange! Ignoring the file {}".format(json_file))
        return
    # Some files have multiple states. It does not make sense!
    if df[df["status"] == "Die Sendung wurde erfolgreich zugestellt."].shape[0] > 1:
        logger.warning("Multiple final states! Ignoring the file {}".format(json_file))
        return
    if df["status"].isnull().any():
        logger.warning("Status column has missing values! Ignoring the file {}".format(json_file))
        return
    if len(df["url"].unique()) > 1:
        logger.warning("Multiple URLs for a single shipment number! Something is wrong")
    if len(df["shipment_number"].unique()) > 1:
        logger.warning("Multiple shipment numbers in the same file. Something is wrong")
    # Ignore the ones which have this status somewhere before the last event. This makes the problem simpler!
    if len(df.index[df["status"] == "Der Empfu00E4nger hat die Sendung in der Filiale abgeholt."].tolist()) > 0:
        if df.index[df["status"] == "Der Empfu00E4nger hat die Sendung in der Filiale abgeholt."].tolist()[0] < len(df):
            return
    if len(df.index[df["status"] == "Die Sendung wurde erfolgreich zugestellt."].tolist()) > 0:
        if df.index[df["status"] == "Die Sendung wurde erfolgreich zugestellt."].tolist()[0] < len(df):
            return
    if len(df.index[df["status"].str.startswith("Die Sendung liegt in der") & df["status"].str.endswith("zur Abholung bereit.")].tolist()) > 0:
        if df.index[df["status"].str.startswith("Die Sendung liegt in der") & df["status"].str.endswith("zur Abholung bereit.")].tolist()[0] < len(df):
            return
    if len(df.index[df["status"] == "Die Sendung in Paketkasten-Anlage eingestellt"].tolist()) > 0:
        if df.index[df["status"] == "Die Sendung in Paketkasten-Anlage eingestellt"].tolist()[0] < len(df):
            return
    df['crawltime'] = pd.to_datetime(df['crawltime'], utc=True)
    df['datum'] = pd.to_datetime(df['datum'], utc=True)
    if df["status"].isnull().values.any():
        logger.warning("Column status has missing values! ShipmentNumber: {}".format(df["shipment_number"].values[0]))
    final_event_array = np.zeros(len(df))
    final_event_array[-1] = 1
    df["is_final_event"] = final_event_array
    return df

In [3]:
def files_to_df(path='../data'):
    """ Convert all files to a single dataframe
    """
    import os
    import pandas as pd
    files = []
    for r, d, f in os.walk(path):
        for file in f:
            if '.json' in file:
                files.append(os.path.join(r, file))
    dfs = [json_to_df(json_file=file) for file in files]
    df = pd.concat(dfs, sort=False)
    return df

In [4]:
df = files_to_df(path='../../data')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


KeyboardInterrupt: 

### Are there any missing values?

In [ ]:
import pandas as pd
def count_missing_values(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
    return missing_value_df
missing_value_df = count_missing_values(df)

In [ ]:
missing_value_df

Column "ort" has many missing values. We fill them with "Unbekannt"

In [ ]:
def fill_missing_values(df):
    """ Fills missing values of specific columns
    in the data frame with pre-defined values.
    """
    df['ort'].fillna("Unbekannt", inplace=True)
    logger.debug("Filled missing values for ort column")
    return df

In [ ]:
df = fill_missing_values(df)

Check again if there are any missing values

In [ ]:
count_missing_values(df)

### Data Stats

In [ ]:
df.shape

Number of unique shipment numbers

In [ ]:
len(df.shipment_number.unique())

### Investigate Data Types

In [ ]:
df.info()

Everything regarding data types seems to be fine! No need for conversion

### Prepare data for exploration and machine learning

- Given the features "datum", "status" and "ort" we want to predict the next "datum" and hopefully also the next "status" and the next "ort"
- We create 3 new columns. "datum_next", "ort_next" and "status_next".

In [ ]:
def create_target_columns(df):
    import numpy as np 

    df['status_next'] = df.groupby(['shipment_number'])['status'].shift(-1)
    df['ort_next'] = df.groupby(['shipment_number'])['ort'].shift(-1)
    df['datum_next'] = df.groupby(['shipment_number'])['datum'].shift(-1)
    # we define values for the case the package is already delivered.
    df.loc[df['is_final_event'] == 1, 'status_next'] = "SuccessDelivery"
    df.loc[df['is_final_event'] == 1, 'ort_next'] = "SuccessDelivery"
    df.loc[df['is_final_event'] == 1, 'datum_next'] = np.datetime64('today')
    
    return df

In [ ]:
# Disable warnings
pd.set_option('mode.chained_assignment', None)
df = create_target_columns(df)

In [ ]:
df.head()

In [ ]:
df.info()

### Data Stats

In [ ]:
count_missing_values(df)

In [ ]:
df.shape

Number of unique shipment numbers

In [ ]:
len(df.shipment_number.unique())

In [ ]:
len(df.status.unique())

In [ ]:
len(df.ort.unique())

### How many different final delivery states exist?

In [ ]:
final_delivery_states = df.groupby(['shipment_number']).tail(1)[["status"]]["status"].unique()
print(final_delivery_states)

### How many different initial delivery states exist?

In [ ]:
initial_delivery_states = df.groupby(['shipment_number']).head(1)[["status"]]["status"].unique()
print(initial_delivery_states)

### Distribution of all states

These are the most common states

In [ ]:
pd.DataFrame(df["status"].value_counts()).head(10)

These are the least common states. What we see here is that there are some states that are basically the same and the following pattern:

"Die Sendung liegt in der ........ zur Abholung bereit"

But the address is each time different. This leads to too many states, which also makes machine learning hard. We ignore the address part and convert all these states to a single one.

In [ ]:
pd.DataFrame(df["status"].value_counts()).tail(10)

In [ ]:
df.loc[(df['status'].str.startswith("Die Sendung liegt in der")) & (df['status'].str.endswith("zur Abholung bereit.")), "status"] = "Die Sendung liegt in der ... zur Abholung bereit." 
df.loc[(df['status'].str.startswith("Die Sendung liegt ab sofort in der Filiale")) & (df['status'].str.endswith("zur Abholung bereit.")), "status"] = "Die Sendung liegt ab sofort in dert Filiale ... zur Abholung bereit."
df.loc[(df['status'].str.startswith("Die Sendung wird zur Abholung in die")) & (df['status'].str.endswith("gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich.")), "status"] = "Die Sendung wird zur Abholung in die ... gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich." 
df.loc[(df['status'].str.startswith("Die Sendung wurde zur Abholung in die")) & (df['status'].str.endswith("gebracht.")), "status"] = "Die Sendung wurde zur Abholung in die ... gebracht." 
df.loc[(df['status'].str.startswith("Die Zustellung am gewu00FCnschten Liefertag")) & (df['status'].str.endswith("wurde gewu00E4hlt.")), "status"] = "Die Zustellung am gewu00FCnschten Liefertag ... wurde gewu00E4hlt." 
df.loc[(df['status'].str.startswith("Die Sendung wird zur Abholung in die")) & (df['status'].str.endswith("gebracht. Die fru00FChestmu00F6gliche Uhrzeit der Abholung kann der Benachrichtigung entnommen werden.")), "status"] = "Die Sendung wird zur Abholung in die ... gebracht. Die fru00FChestmu00F6gliche Uhrzeit der Abholung kann der Benachrichtigung entnommen werden." 


df.loc[(df['status_next'].str.startswith("Die Sendung liegt in der")) & (df['status_next'].str.endswith("zur Abholung bereit.")), "status_next"] = "Die Sendung liegt in der ... zur Abholung bereit." 
df.loc[(df['status'].str.startswith("Die Sendung liegt ab sofort in der Filiale")) & (df['status'].str.endswith("zur Abholung bereit.")), "status"] = "Die Sendung liegt ab sofort in dert Filiale ... zur Abholung bereit."
df.loc[(df['status_next'].str.startswith("Die Sendung wird zur Abholung in die")) & (df['status_next'].str.endswith("gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich.")), "status_next"] = "Die Sendung wird zur Abholung in die ... gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich." 
df.loc[(df['status_next'].str.startswith("Die Sendung wurde zur Abholung in die")) & (df['status_next'].str.endswith("gebracht.")), "status_next"] = "Die Sendung wurde zur Abholung in die ... gebracht." 
df.loc[(df['status_next'].str.startswith("Die Zustellung am gewu00FCnschten Liefertag")) & (df['status_next'].str.endswith("wurde gewu00E4hlt.")), "status_next"] = "Die Zustellung am gewu00FCnschten Liefertag ... wurde gewu00E4hlt." 
df.loc[(df['status_next'].str.startswith("Die Sendung wird zur Abholung in die")) & (df['status_next'].str.endswith("gebracht. Die fru00FChestmu00F6gliche Uhrzeit der Abholung kann der Benachrichtigung entnommen werden.")), "status_next"] = "Die Sendung wird zur Abholung in die ... gebracht. Die fru00FChestmu00F6gliche Uhrzeit der Abholung kann der Benachrichtigung entnommen werden." 

In [ ]:
pd.DataFrame(df["status"].value_counts()).head(10)

In [ ]:
pd.DataFrame(df["status"].value_counts()).tail(10)

In [ ]:
len(df["status"].unique())

In [ ]:
len(df["status_next"].unique())

### Distribution of the locations

In [ ]:
len(df["ort"].unique())

In [ ]:
pd.DataFrame(df["ort"].value_counts()).head(20)

In [ ]:
pd.DataFrame(df["ort"].value_counts()).tail(20)

Notice: One idea here could be to merge some locations. 

### Remove unused columns

In [ ]:
df.drop(columns=["crawltime", "url"], inplace=True)

In [ ]:
df.head()

### Time diff between events

In [ ]:
df['datum'] = pd.to_datetime(df['datum'], utc=True)
df['datum_next'] = pd.to_datetime(df['datum_next'], utc=True)
df["datum_diff"] = df["datum_next"] - df["datum"]

Notice that for is_final_event=1, the datum_diff does not make any sense. Do not use it in the analysis

In [ ]:
df.head()

### Define the problem as a regression problem

In [ ]:
df["datum_diff_h"] = df['datum_diff'].astype('timedelta64[h]')

This means now that we want to predict the time between two events. The column "datum_diff_h" tells us the time diff betwen the two events in hours

##### First we only use two features. So the question is, given status and ort of an event, can we predict, when the next event will happen?

In [ ]:
df_temp = df[df["is_final_event"] == 0]
X = df_temp[["status", "ort"]]
y = df_temp["datum_diff_h"]

Notice that "status" and "ort" are categorical variables. We shoud encode them!

In [ ]:
X["status"] = X["status"].astype('category')
X["status_code"] = X["status"].cat.codes
X["ort"] = X["ort"].astype('category')
X["ort_code"] = X["ort"].cat.codes

Create trainig and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

regr = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=0)
regr.fit(X_train[["status_code", "ort_code"]], y_train)

##### Coefficient of determination R^2 of the prediction with cross validation

In [ ]:
cross_val_score(regr, X_train[["status_code", "ort_code"]], y_train, cv=10)

##### Feature Importance

In [ ]:
print(regr.feature_importances_)

"status" played a bigger role in predictions comparing to "ort"

##### Fit the model and test it on the test set

In [ ]:
model = regr.fit(X_train[["status_code", "ort_code"]], y_train)

In [ ]:
y_pred = model.predict(X_test[["status_code", "ort_code"]])

##### Metrics

In [ ]:
from sklearn import metrics
import numpy as np
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

##### Distribution of errors

In [ ]:
import seaborn as sns
sns.distplot(y_test-y_pred, hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 1})

### For which data point do I have a minimal error?

In [ ]:
X_test_df = pd.DataFrame(X_test).reset_index(drop=True)
y_test_df = pd.DataFrame(y_test).reset_index(drop=True)
y_pred_df = pd.DataFrame(y_pred).reset_index(drop=True)

In [ ]:
prediction_df = pd.concat([X_test_df, y_test_df, y_pred_df], axis=1)

In [ ]:
prediction_df.head()

In [ ]:
prediction_df.columns = ['status', 'ort', 'status_code', 'ort_code', 'y', 'y_pred']

In [ ]:
prediction_df.head()

In [ ]:
prediction_df["diff"] = abs(prediction_df["y"] - prediction_df["y_pred"])

In [ ]:
prediction_df.head()

In [ ]:
prediction_df[prediction_df["diff"] > 25].values